<a href="https://colab.research.google.com/github/mzambetti01/Bayesian-Netowrk/blob/main/ECE603_Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is my code for ECE 603: Project 2 - Spam Filter

## Naive Bayesian Network

Matt Zambetti

10/10/22

Done in Google Colab (Python)

In this code I have made a simple Bayesian Network that identifies whether an email is spam or not based on the words it contains.

In [ ]:
# Import Section
# Importing matfiles was done using: https://towardsdatascience.com/how-to-load-matlab-mat-files-in-python-1f200e1287b5
from scipy.io import loadmat
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Getting our data set up
spamdata = loadmat('/content/drive/My Drive/spamdata.mat')
X = spamdata['X']
X2 = spamdata['X2']
Y = spamdata['y']
Y2 = spamdata['y2']
# Our smoothing constant
k=1

Here we are using 3 dictionaries.
1. P_X is P(X), the probability distribution for each word. 

  Given X_i = [x_0, x_1, ... , x_n] 

  P(X) is the probability of x_i where 0 <= i <=n

  To store this we are using a dictionary where the key value is the index and the value is its probability of being in an email or not.

2. P_Y is P(Y), the probability of the email being spam or not.

  To store this value we are not going to store the actual probability but rather the magnitude of each set. So, the first value is the number of emails that were not spam, the second is the number of emails that are spam, and the third is the total number of emails, both spam and not.

3. P_XY is P(X|Y), the probability a word is in an email, given it spam or not.

  This value is also using a dictionary, where the key is the index of the word and the value is a pair of numbers. The first being the probability of emails that were not spam and contained the word and the second being the probability of emails that were spam and contained the word.

In [ ]:
# Getting our P(X), P(Y), P(X|Y)
P_X = dict()                      # P(X) = {x_index, prob}
P_Y = [0, 0, len(Y)]              # P(Y) = [not, spam, total Y values]
P_XY = dict()                     # P(X|Y) = {X, [Y=0, Y=1]}

# Creating our dictionary to keep track of indices and their values of Y
index_dict = dict()
for idx in range(len(X[1])):
  index_dict[idx] = [0,0]

# Iterating over all values of X
for idx in range(len(X)):
  X_i = list(X[idx])
  
  # Keeping track of our y values
  if Y[idx] == 1:
    P_Y[1] +=1
  else:
    P_Y[0] +=1

  # Going over each element of X
  for x_idx in range(len(X_i)):
    x = X_i[x_idx]
    # If this word appears figure out whether its spam or not
    if x == 1:
      if Y[idx] == 1:
        index_dict[x_idx][1] += 1
      else:
        index_dict[x_idx][0] += 1

# Getting P(X) and P(X|Y)
for key in index_dict.keys():
  # Our P(X) = |spam and not| / |X|
  P_X[key] = sum(index_dict[key]) / len(X);

  # Getting our X|Y distr
  # Also smoothing our data
  # P(X|Y) = P(X & Y) + k / P(Y) + 2 * k
  P_XY[key] = [(index_dict[key][0] + k) / (P_Y[0] + (2*k)), (index_dict[key][1] + k) / (P_Y[1] + (2 * k))];
  


Here we will test our data.

To calculate the probabilites of being spam or not we will use the Naive Bayesian idea.

This means calculate P(X|Y=y1) = P(X=x_i|Y=y_1) x P(X=x_2|Y=y_1) x ... x P(X=x_m|Y=y_1)

In [ ]:
# Start our with defining some variables we will need throughout this section
correct = 0

# Going through our test values
for idx in range(len(X2)):

  # Grab the current entry we want to test
  X_test = list(X2[idx])

  # Let p_xy0 be P(X|Y=0)
  p_xy0 = 1
  # Let p_xy1 be P(X|Y=1)
  p_xy1 = 1

  # Iterating over each value in X_test
  for x_idx in range(len(X_test)):
    
    x = X_test[x_idx]

    # If the word is present in the email
    if x == 1:
      # Update the conditional probabilities
      p_xy0 *= P_XY[x_idx][0]
      p_xy1 *= P_XY[x_idx][1]
  # Multiply by P(Y) to get the final probabilities
  prob_spam = p_xy1 * (P_Y[1]/P_Y[2])
  prob_not = p_xy0 * (P_Y[0]/P_Y[2])

  # Now compare the two
  if prob_spam > prob_not:
    temp_y = 1
  else:
    temp_y = 0

  # Checking to see if we have the correct value
  if temp_y == Y2[idx][0]:
    correct += 1

# Printing our correctness
print(correct/len(X2))

0.936
